In [ ]:
versioninfo()

In [ ]:
x = [1, 2, 3]

In [ ]:
x = [1 2; 3 4]

In [ ]:
x = rand(5, 3)

In [ ]:
x = ones(5, 3)

In [ ]:
x = zeros(5, 3)

In [ ]:
x = randn(5, 3)

In [ ]:
x = rand(BigFloat, 5, 3)

In [ ]:
x = rand(Float32, 5, 3)

In [ ]:
length(x)

In [ ]:
size(x)

In [ ]:
x

In [ ]:
x[2, 3]

In [ ]:
x[:, 3]

In [ ]:
x + x

In [ ]:
x - x

In [ ]:
x .+ 1

In [ ]:
zeros(5, 8) .+ (1:5)

In [ ]:
zeros(5, 8) .+ (1:8)'

In [ ]:
W = randn(5, 10)
x = rand(10)
W * x

In [ ]:
f(x) = 3x^2 +2x +1

In [ ]:
f(5)

In [ ]:
using Flux: gradient

In [ ]:
df(x) = gradient(f, x)[1]

In [ ]:
df(5)

In [ ]:
ddf(x) = gradient(df, x)[1]

In [ ]:
ddf(5)

In [ ]:
mysin(x) = sum((-1)^k * x^(1+2k)/factorial(1+2k) for k = 0:5)

In [ ]:
mysin(π/2)

In [ ]:
x = 0.50

In [ ]:
mysin(x), gradient(mysin, x)

In [ ]:
sin(x), cos(x)

In [ ]:
myloss(W, x, b) = sum(W * x .+ b)

In [ ]:
W = randn(3, 5)
b = zeros(3)
x = rand(5)

In [ ]:
d = gradient(myloss, W, x, b)

In [ ]:
x

In [ ]:
W

In [ ]:
b

In [ ]:
d[1]

In [ ]:
d[2]

In [ ]:
d[3]

In [ ]:
sum(W, dims=1)

In [ ]:
using Flux: params

In [ ]:
W = randn(3, 5)
b = zeros(3)
x = rand(5)

In [ ]:
y(x) = sum(W * x .+ b)

In [ ]:
grads = gradient(() -> y(x), params([W, b]))

In [ ]:
grads[W]

In [ ]:
grads[b]

In [ ]:
using Flux

In [ ]:
m = Dense(10, 5)

In [ ]:
x =  rand(Float32, 10)

In [ ]:
params(m)

In [ ]:
x = rand(Float32, 10)
m = Chain(Dense(10, 5, relu), Dense(5, 2), softmax)

In [ ]:
l(x) = sum(Flux.crossentropy(m(x), [0.50, 0.50]))

In [ ]:
grads = gradient(params(m)) do
    l(x)
end

In [ ]:
for p in params(m)
    println(size(p), "=>", grads[p], "\n")
end

In [ ]:
using Flux.Optimise: update!, Descent

In [ ]:
η = 0.10

In [ ]:
for p in params(m)
    update!(p, -η * grads[p])
end

In [ ]:
opt = Descent(0.10)

In [ ]:
data, labels = rand(10, 100), fill(0.50, 2, 100)

In [ ]:
loss(x, y) = Flux.crossentropy(m(x), y)

In [ ]:
Flux.train!(loss, params(m), [(data, labels)], opt)

# Training a classifier

In [ ]:
using Statistics

In [ ]:
using Zygote

In [ ]:
using Flux, Flux.Optimise

In [ ]:
using Metalhead, Images

In [ ]:
using Metalhead: trainimgs

In [ ]:
using Images.ImageCore

In [ ]:
using Flux: onehotbatch, onecold

In [ ]:
using Base.Iterators: partition

In [ ]:
Metalhead.download(CIFAR10)

In [ ]:
X = trainimgs(CIFAR10);

In [ ]:
labels = onehotbatch([X[i].ground_truth.class for i in 1:length(X)], 1:10);

In [ ]:
image(x) = x.img
ground_truth(x) = x.ground_truth
image.(X[rand(1:end, 10)])

In [ ]:
getarray(X) = float.(permutedims(channelview(X), (2, 3, 1)))
imgs = [ getarray(x.img) for x in X ];

In [ ]:
trainset = 1:49000
valset = 49001:50000
train = ([(cat(imgs[i]..., dims=4), labels[:, i]) for i in partition(trainset, 1000)]) |> gpu;
valX = cat(imgs[valset]..., dims=4) |> gpu;
valY = labels[:, valset] |> gpu;

In [ ]:
m = Chain(
    Conv((5, 5), 3=>16, relu),
    MaxPool((2, 2)),
    Conv((5, 5), 16=>8, relu),
    MaxPool((2, 2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(200, 120),
    Dense(120, 84),
    Dense(84, 10),
    softmax
) |> gpu

In [ ]:
using Flux: crossentropy, Momentum

In [ ]:
loss(x, y) = crossentropy(m(x), y)

In [ ]:
opt = Momentum(0.01)

In [ ]:
accuracy(x, y) = mean(onecold(m(x), 1:10) .== onecold(y, 1:10))

In [ ]:
epochs = 3

for epoch ∈ 1:epochs
    for d ∈ train
        gs = gradient(params(m)) do
            l = loss(d...)
        end
        update!(opt, params(m), gs)
    end
    @show accuracy(valX, valY)
end

## Testing the network

In [ ]:
valset = valimgs(CIFAR10);

In [ ]:
valimg = [getarray(valset[i].img) for i in 1:10000];
labels = onehotbatch([valset[i].ground_truth.class for i in 1:10000], 1:10);
test = gpu.([(cat(valimg[i]..., dims=4), labels[:, i]) for i in partition(1:10000, 1000)]);

In [ ]:
ids = rand(1:10000, 10)
image.(valset[ids])

In [ ]:
rand_test = getarray.(image.(valset[ids]))
rand_test = cat(rand_test..., dims=4) |> gpu
rand_truth = ground_truth.(valset[ids])
m(rand_test)

In [ ]:
accuracy(test[1]...)

In [ ]:
class_correct = zeros(10)
class_total = zeros(10)

In [ ]:
for i ∈ 1:10
    preds = m(test[i][1])
    label = test[i][2]
    for j ∈ 1:1000
        pred_class = findmax(preds[:, j])[2]
        actual_class = findmax(label[:, j])[2]
        if pred_class == actual_class
            class_correct[pred_class] += 1
        end
        class_total[actual_class] += 1
    end
end

In [ ]:
@show class_correct ./ class_total